In [1]:
import pandas as pd
import pymongo
import os
from pymongo import MongoClient
import pprint

In [2]:
client = MongoClient(
    host = "127.0.0.1",
    port = 27017,
    username = "antoine",
    password = "pela"
)

print(client.list_database_names())

['admin', 'config', 'local', 'reco_movies']


In [3]:
db = client['reco_movies']

In [4]:
# Lister et afficher les collections
collections = db.list_collection_names()
print("Collections dans la base de données :")
for collection in collections:
    print(collection)

Collections dans la base de données :
movies
links
ratings


### CHARGEMENT DES DONNEES DANS NOS COLLECTIONS

In [5]:
# CREATION COLLECTION MOVIES
collection_movies = db['movies']

movies = pd.read_csv('/home/antoine/Ml_Ops_Movies_Reco/src/data/data/raw/movies.csv')

# Split sur les pipes
movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))
# Extraction de l'année et mise à jour du titre
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')[0]
movies['title'] = movies['title'].str.replace(r' \(\d{4}\)', '', regex=True)
movies.ffill(inplace= True)
movies.head()


,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [8]:
records_movies =movies.to_dict(orient="records")

collection_movies.insert_many(records_movies)

InsertManyResult([ObjectId('671a8b9b4cc88095686937ce'), ObjectId('671a8b9b4cc88095686937cf'), ObjectId('671a8b9b4cc88095686937d0'), ObjectId('671a8b9b4cc88095686937d1'), ObjectId('671a8b9b4cc88095686937d2'), ObjectId('671a8b9b4cc88095686937d3'), ObjectId('671a8b9b4cc88095686937d4'), ObjectId('671a8b9b4cc88095686937d5'), ObjectId('671a8b9b4cc88095686937d6'), ObjectId('671a8b9b4cc88095686937d7'), ObjectId('671a8b9b4cc88095686937d8'), ObjectId('671a8b9b4cc88095686937d9'), ObjectId('671a8b9b4cc88095686937da'), ObjectId('671a8b9b4cc88095686937db'), ObjectId('671a8b9b4cc88095686937dc'), ObjectId('671a8b9b4cc88095686937dd'), ObjectId('671a8b9b4cc88095686937de'), ObjectId('671a8b9b4cc88095686937df'), ObjectId('671a8b9b4cc88095686937e0'), ObjectId('671a8b9b4cc88095686937e1'), ObjectId('671a8b9b4cc88095686937e2'), ObjectId('671a8b9b4cc88095686937e3'), ObjectId('671a8b9b4cc88095686937e4'), ObjectId('671a8b9b4cc88095686937e5'), ObjectId('671a8b9b4cc88095686937e6'), ObjectId('671a8b9b4cc88095686937

In [ ]:
# CREATION COLLECTION RATINGS
collection_ratings = db['ratings']

ratings = pd.read_csv('/home/antoine/Ml_Ops_Movies_Reco/src/data/data/raw/ratings.csv')

ratings = ratings.drop('timestamp', axis = 1)

records_ratings = ratings.to_dict(orient="records")

collection_ratings.insert_many(records_ratings)

In [7]:
# CREATION COLLECTION LINKS
collection_links = db['links']

links = pd.read_csv('/home/antoine/Ml_Ops_Movies_Reco/src/data/data/raw/links.csv')

records_links = links.to_dict(orient="records")

collection_links.insert_many(records_links)

InsertManyResult([ObjectId('671a8fef3672d958e54eaf7d'), ObjectId('671a8fef3672d958e54eaf7e'), ObjectId('671a8fef3672d958e54eaf7f'), ObjectId('671a8fef3672d958e54eaf80'), ObjectId('671a8fef3672d958e54eaf81'), ObjectId('671a8fef3672d958e54eaf82'), ObjectId('671a8fef3672d958e54eaf83'), ObjectId('671a8fef3672d958e54eaf84'), ObjectId('671a8fef3672d958e54eaf85'), ObjectId('671a8fef3672d958e54eaf86'), ObjectId('671a8fef3672d958e54eaf87'), ObjectId('671a8fef3672d958e54eaf88'), ObjectId('671a8fef3672d958e54eaf89'), ObjectId('671a8fef3672d958e54eaf8a'), ObjectId('671a8fef3672d958e54eaf8b'), ObjectId('671a8fef3672d958e54eaf8c'), ObjectId('671a8fef3672d958e54eaf8d'), ObjectId('671a8fef3672d958e54eaf8e'), ObjectId('671a8fef3672d958e54eaf8f'), ObjectId('671a8fef3672d958e54eaf90'), ObjectId('671a8fef3672d958e54eaf91'), ObjectId('671a8fef3672d958e54eaf92'), ObjectId('671a8fef3672d958e54eaf93'), ObjectId('671a8fef3672d958e54eaf94'), ObjectId('671a8fef3672d958e54eaf95'), ObjectId('671a8fef3672d958e54eaf

### VISUALISATION DES COLLECTIONS

In [6]:
ratings = db['ratings']

print(ratings.find_one())

{'_id': ObjectId('671a9b3b0f9ef42264412d63'), 'userId': 1, 'movieId': 2, 'rating': 3.5}


In [7]:
# Compter le nombre total de documents
nombre_documents = ratings.count_documents({})
print("Nombre total de documents :", nombre_documents)

Nombre total de documents : 20300263


In [8]:
movies = db.movies

pprint.pprint(movies.find_one())

{'_id': ObjectId('671a8b9b4cc88095686937ce'),
 'genres': ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 'movieId': 1,
 'title': 'Toy Story',
 'year': '1995'}


In [9]:
# Compter le nombre total de documents
nombre_movies = movies.count_documents({})
print("Nombre total de movies :", nombre_movies)

Nombre total de movies : 27278


### EXTRACTION DES DONNEES

In [15]:
# Étape 1 : Récupérer tous les movieId pour userId = 1
user_id = 1
ratings_collection = db["ratings"]  # Remplacez par le nom de votre collection de notations
movie_ids = ratings.find({"userId": user_id}, {"movieId": 1})

# Extraire les movieId dans une liste
movie_id_list = [doc["movieId"] for doc in movie_ids]

# Étape 2 : Récupérer les titres des films correspondants
movies_collection = db["movies"]  # Remplacez par le nom de votre collection de films
titles = movies_collection.find({"movieId": {"$in": movie_id_list}}, {"title": 1})

# Afficher les titres des films
for title in titles:
    print(title["title"])

Jumanji (1995)
City of Lost Children, The (CitÃ© des enfants perdus, La) (1995)
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Seven (a.k.a. Se7en) (1995)
Usual Suspects, The (1995)
Rumble in the Bronx (Hont faan kui) (1995)
Rob Roy (1995)
Clerks (1994)
Interview with the Vampire: The Vampire Chronicles (1994)
Star Wars: Episode IV - A New Hope (1977)
LÃ©on: The Professional (a.k.a. The Professional) (LÃ©on) (1994)
Pulp Fiction (1994)
Shawshank Redemption, The (1994)
What's Eating Gilbert Grape (1993)
Mask, The (1994)
Blade Runner (1982)
Terminator 2: Judgment Day (1991)
Silence of the Lambs, The (1991)
Dragonheart (1996)
Wizard of Oz, The (1939)
2001: A Space Odyssey (1968)
Escape to Witch Mountain (1975)
Die Hard (1988)
Fish Called Wanda, A (1988)
Monty Python's Life of Brian (1979)
Reservoir Dogs (1992)
Platoon (1986)
E.T. the Extra-Terrestrial (1982)
Monty Python and the Holy Grail (1975)
One Flew Over the Cuckoo's Nest (1975)
Star Wars: Episode V - The Empire Strikes Back (1980)
Raider

In [16]:
# Étape 1 : Récupérer tous les movieId pour userId = 5
user_id = 5
ratings_collection = db["ratings"]  # Remplacez par le nom de votre collection de notations
excluded_movie_ids = ratings_collection.find({"userId": user_id}, {"movieId": 1})

# Extraire les movieId dans une liste
excluded_movie_id_list = [doc["movieId"] for doc in excluded_movie_ids]

# Étape 2 : Récupérer tous les films sauf ceux exclus
movies_collection = db["movies"]  # Remplacez par le nom de votre collection de films
movies_cursor = movies_collection.find({"movieId": {"$nin": excluded_movie_id_list}}, {"_id": 0, "movieId": 1, "title": 1})

# Convertir le curseur en liste et supprimer les doublons
movies_list = list(movies_cursor)

# Étape 3 : Récupérer les ratings associés à ces films, y compris userId
movie_ids_for_ratings = [movie["movieId"] for movie in movies_list]
ratings_cursor = ratings_collection.find({"movieId": {"$in": movie_ids_for_ratings}})

# Convertir les ratings en liste
ratings_list = list(ratings_cursor)

# Étape 4 : Créer un DataFrame avec les films et leurs ratings
movies_df = pd.DataFrame(movies_list)
ratings_df = pd.DataFrame(ratings_list)

# Fusionner les deux DataFrames sur movieId
merged_df = pd.merge(movies_df, ratings_df, on="movieId", how="left")

# Afficher le DataFrame final avec userIds, titles et ratings
print(merged_df)

: 